In [1]:
import sys
sys.path.append("../../src/models")

In [2]:
from simulate_pandemic import init_infection, spread_infection, lambda_leak_expose, update_population
from simulate_pandemic import main
from actions import city_restrictions

In [3]:
from numpy.random import default_rng
import pandas as pd
from datetime import datetime
from tqdm import tqdm

In [4]:
from joblib import Parallel, delayed

In [9]:
def make_df(week, sim, action, data):
    df = pd.DataFrame(pd.Series(data[:, 1]).value_counts()).T
    df['simulation'] = sim
    df['week'] = week
    df['action'] = action
    
    return df

In [10]:
rng = default_rng(None)

gpickle_path = "../../data/processed/SP_multiGraph_Job_Edu_Level.gpickle"

prhome = 0.06
p_r = {
    'home'    :  prhome,
    'neighbor':  .1*prhome,
    'work'    :  .1*prhome,
    'school'  :  .15*prhome,
}

In [16]:
def simulate_and_save(it, folder = 'generated_sims/'):
    policy = rng.choice(list(city_restrictions.keys()), size=int(350/7), replace=True, shuffle=True)
    data, pop_matrix = main(gpickle_path=gpickle_path, p_r=p_r, policy=policy, disable_tqdm=True)
    weekly_data = [data[i] for i in range(int(350/7))]
    time = datetime.now().strftime("%Y%m%d%H%M%S")
    df = pd.concat([make_df(week, time, action, data)
                         for week, (data, action) in enumerate(zip(weekly_data, policy))])
    df.to_parquet(folder + f'simulation_{time}{str(it)}.parquet')
    return time+str(it), df

In [17]:
res = Parallel(n_jobs=1)(delayed(simulate_and_save)(i) for i in tqdm(range(1)))

100%|██████████| 1/1 [00:12<00:00, 12.69s/it]


,0,-1,2,1,3
0,2768866,3528,1172,974,60


In [27]:
pd.DataFrame(res[0][1][['node_id', 'status']]['status'].value_counts()).T

,0,-1,2,1,3
status,2768866,3528,1172,974,60
